## 讀檔 + 切詞

In [6]:
# -*- coding: utf-8 -*-
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
import jieba
import jieba.analyse
import numpy as np

jieba.set_dictionary('dict.txt.big')##加入繁體中文

totalMovie = 6000
Seg = [] #切詞後的結果
labels = [] 
labelType = []

def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

with open('movie.csv', encoding="utf-8") as file:
    stopwords = stopwordslist('StopWords.txt')  # 這裏加載停用詞的路徑 
    for i,line in enumerate(file.readlines()):
        
        if i == totalMovie:
            break
        storyText = line.split(",")
        story  = storyText[1]
        if storyText[2] == 'NA':
            continue
        temp_text = re.findall('[\u4e00-\u9fa5]', story)
        story = ''.join(temp_text)
        
        jieba.add_word('為什麼')
        jieba.add_word('有沒有')
         
        tags = jieba.lcut(story,cut_all=False)
        ThisLine = ''
        for tag in tags:
            if tag not in stopwords:  
                if tag != '\t' and tag != '\n' and tag != '' and len(tag) > 1:  
                    ThisLine += tag  
                    ThisLine += " "   #再次組合成【帶空格】的串
                    
        Seg.append(ThisLine)
        
        label = storyText[1].split(" ")
        labels.append(label[0])

labelType = list(set(labels))
print(labelType)
print(len(labelType))
print(len(Seg))
print("Finish")

Building prefix dict from /Users/chiao/summer_data/HW3/dict.txt.big ...
Loading model from cache /var/folders/w2/vcklh68j4wbblf0hwp6w_wk40000gn/T/jieba.u519da0ee965d36f911b06d3d932c871e.cache
Loading model cost 1.069 seconds.
Prefix dict has been built successfully.


IndexError: list index out of range

## TF-IDF

In [3]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(Seg))
#獲取詞袋模型中的所有詞語  
word=vectorizer.get_feature_names()
print("Finish")

Finish


## TSVD降低維度

In [134]:
from sklearn.decomposition import TruncatedSVD

n_pick_topics = 20          
svd = TruncatedSVD(n_pick_topics)               
SVD = svd.fit_transform(tfidf)
print("Finish")

Finish


## KNN模型

In [226]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import random

Max = 0
MaxScore = 0
totalScore = 0
for i in range(0, 100):
    r = random.randrange(100,1000)
    train_data , test_data , train_label , test_label = train_test_split(SVD,labels, test_size=500, random_state=r)
    knn = KNeighborsClassifier(n_neighbors=17, p=2, weights='distance')
    knn.fit(train_data,train_label)
    Result = knn.predict(test_data)
    Score = accuracy_score(test_label, Result)
    totalScore += Score
    if Score > MaxScore:
        MaxScore = Score
        Max = r
        
print("Finish")
#平均成績
print("\n正確率：", MaxScore*100, '%') 

Finish

正確率： 64.0 %


## 預測結果

In [69]:
#最高紀錄，曾經有過...
print( "正確率：", MaxScore*100, '%') #1986348

正確率： 80.0 %
